In [1]:
import sys
import pygame
from pygame.locals import *
import pymunk 

from pymunk.vec2d import Vec2d

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Loading chipmunk for Darwin (64bit) [/Users/alecx/anaconda3/envs/physics/lib/python2.7/site-packages/pymunk/libchipmunk.dylib]


In [2]:
from pymunk import pygame_util
import random
import math

In [3]:
# Converts a pixel length to a meter length: 100px = 1m
def px2m(length):
    return length/100.0

# Converts kinetic energy calculated from pixels to kinetic energy calculated from meters
def px2e(ke):
    ke /= 2.0 # Pymunk leaves out 1/2 in KE=(1/2)mv^2 calculation
    
    return ke/10000.0

In [4]:
def add_bullet(space, b_mass=1):
    radius = 10
    moment = pymunk.moment_for_circle(b_mass, 0, radius) # 1
    body = pymunk.Body(b_mass, moment) # 2
    body.position = 0, 200 # 3
    body.start_position = Vec2d(body.position)
    shape = pymunk.Circle(body, radius) # 4
    
    shape.elasticity = 0
    shape.friction = 0
    shape.collision_type = 1
    
    shape.color = pygame.color.THECOLORS["black"]
    
    space.add(body, shape) # 5
    return shape

In [5]:
def add_pendulum(space, b_mass=50):
    
    # x and y coords for block's center
    center_x = 300
    center_y = 100
    
    # Add pendulum's block
    #block_mass = 10000    
    #block_moment = pymunk.moment_for_box(block_mass, (200,100))
    block_body = pymunk.Body()
    
    block_body.position = center_x, center_y
    block_shape = pymunk.Poly(block_body, [(center_x-100, center_y-50),(center_x-100, center_y+50),(center_x+100, center_y+50),(center_x+100, center_y-50)])
    block_shape.density = b_mass/(200.0*100)
    
    block_shape.elasticity = 0
    block_shape.friction = 0
    block_shape.collision_type = 2
    
    block_shape.color = pygame.color.THECOLORS["gray"]
    
    space.add(block_body, block_shape)    
    
    
    # Make pendulum hang from two strings
    pivot_point_1 = pymunk.Body(body_type = pymunk.Body.STATIC)
    pivot_point_1.position = (center_x, center_y+100)
    
    pivot_point_2 = pymunk.Body(body_type = pymunk.Body.STATIC)
    pivot_point_2.position = (center_x, center_y+100)
    
    joint_1 = pymunk.constraint.PinJoint(pivot_point_1, block_body, (center_x-100, center_y+100), (center_x-100,center_y+50))
    joint_2 = pymunk.constraint.PinJoint(pivot_point_2, block_body, (center_x+100, center_y+100), (center_x+100,center_y+50))
    
    space.add(joint_1, joint_2)
    
    return block_shape

In [6]:
def bullet_hits(arbiter, space, _):    
    '''
    This function is called after the bullet finishes transferring momentum to block (during post_solve)
    '''
    # Transfer mass of bullet to block
    space.bodies[1].mass += (space.bodies[0].mass)
    
    
    space.remove(space.shapes[0]) # remove the bullet shape
    space.remove(space.bodies[0]) # remove the bullet body
    

In [7]:
class simulation:
    def __init__(self):
        self.screen_width, self.screen_height = 800, 600 # Equivalent to 8m x 6m
        
        self.max_height_reached = None
        self.max_potential_energy = None
    
    def start_simulation(self):
        pygame.init()
        self.myfont = pygame.font.SysFont('Comic Sans MS', 24)

        self.screen = pygame.display.set_mode((self.screen_width, self.screen_height))
        self.screen.set_alpha(None)
        
        pygame.display.set_caption("Ballistic Pendulum Simulation")
        self.clock = pygame.time.Clock()

        self.background = pygame.Surface(self.screen.get_size())
        self.background = self.background.convert()
        self.background.fill((255, 255, 255))

        self.space = pymunk.Space()
        self.space.gravity = 0, -980.665 # Equivalent to ~9.8 m/s^2
        self.space.damping = 1

        self.bullet = add_bullet(self.space)
        self.bullet_mass = self.space.bodies[0].mass # store bullet mass for future calculations
        self.block = add_pendulum(self.space)
        self.impulse = 15000

        # Setup bullet-block collision callback function
        self.h = self.space.add_collision_handler(1, 2) # ...(COLLTYPE_BULLET, COLLTYPE_BLOCK)
        self.h.post_solve = bullet_hits

        self.reached_apex = False # set flag for re-calculating block's potential energy

        self.draw_options = pymunk.pygame_util.DrawOptions(self.screen)
        self.draw_options.constraint_color = pygame.color.THECOLORS["brown"]
        self.draw_options.collision_point_color = pygame.color.THECOLORS["red"] 

        # Equivalent to 150 kgm/s (with default ball mass)
        self.space.shapes[0].body.apply_impulse_at_local_point((self.impulse,0)) 

    def run_simulation(self):
        
        self.screen.blit(self.background, (0, 0))
        
        # One-time Calculations (when pendulum reaches max-height)
        if(px2e(self.block.body.kinetic_energy) < 3 and len(self.space.bodies)==1 and self.reached_apex==False):
            # Calculate E = mgh
            self.max_height_reached = px2m(self.block.body.position[1] - 100)
            self.max_potential_energy = self.block.body.mass*px2m(self.space.gravity[1]*-1)*px2m(self.block.body.position[1] - 100)
            
            # Calculate vf from mgh = (1/2)mv^2
            self.maxv_from_mgh = math.sqrt(self.max_potential_energy * 2 / self.block.body.mass)
            
            # Calculate vi from m(vi) = (m+M)(vf)
            # vi = (m+M)/m * (vf)
            self.bulletv_from_maxv = (self.block.body.mass)/self.bullet_mass * self.maxv_from_mgh
        
            self.reached_apex = True
                
        
        # Ongoing Calculations
        self.text_h = self.myfont.render("Block Height: %.2f m" % px2m(self.block.body.position[1] - 100), True, (0, 0, 0), (255,255,255))
        self.screen.blit(self.text_h, (self.screen_width/2-self.text_h.get_rect().width/2, self.screen_height/2 - 200))

        self.text_ke = self.myfont.render("Block KE: %.2f J" % px2e(self.block.body.kinetic_energy), True, (0, 0, 0), (255,255,255))
        self.screen.blit(self.text_ke, (self.screen_width/2-self.text_ke.get_rect().width/2, self.screen_height/2 - 225))

        if(self.max_height_reached != None):
            self.text_maxh = self.myfont.render("Max Height Reached: %.2f m" % self.max_height_reached, True, (0, 0, 0), (255,255,255))
            self.screen.blit(self.text_maxh, (self.screen_width/2-self.text_maxh.get_rect().width/2, self.screen_height/2 + 175))
            
            self.text_mgh = self.myfont.render("Max PE Reached: %.2f J" % self.max_potential_energy, True, (0, 0, 0), (255,255,255))
            self.screen.blit(self.text_mgh, (self.screen_width/2-self.text_mgh.get_rect().width/2, self.screen_height/2 + 200))    

            self.text_maxv = self.myfont.render("Calculated Max Block Velocity: %.2f m/s" % self.maxv_from_mgh, True, (0,0,0), (255,255,255))
            self.screen.blit(self.text_maxv, (self.screen_width/2-self.text_maxv.get_rect().width/2, self.screen_height/2 + 225))

            self.text_bulletv = self.myfont.render("Calculated Bullet Velocity: %.2f m/s" % self.bulletv_from_maxv, True, (0,0,0), (255,255,255))
            self.screen.blit(self.text_bulletv, (self.screen_width/2-self.text_bulletv.get_rect().width/2, self.screen_height/2 + 250))
        
            self.text_truev = self.myfont.render("True Bullet Velocity: %.2f m/s" % px2m(self.impulse/self.bullet_mass), True, (255,0,0), (255,255,255))
            self.screen.blit(self.text_truev, (self.screen_width/2-self.text_truev.get_rect().width/2, self.screen_height/2 + 275))
        
        
        self.space.step(1/200.0) # TEMP: Slow-motion
        
        self.space.debug_draw(self.draw_options)


        pygame.display.flip()
        self.clock.tick(50)

    
    def restart_simulation(self, mass_i_bullet=1, mass_i_block=50):        
    
        numShapes = len(self.space.shapes)
        for i in range(numShapes):
            self.space.remove(self.space.shapes[0])
            
        numBodies = len(self.space.bodies)
        for i in range(numBodies):
            self.space.remove(self.space.bodies[0])      
        
        numConstraints = len(self.space.constraints)
        for i in range(numConstraints):
            self.space.remove(self.space.constraints[0])
        
        self.bullet = add_bullet(self.space, mass_i_bullet)
        self.block = add_pendulum(self.space, mass_i_block)
        
        self.draw_options = pymunk.pygame_util.DrawOptions(self.screen)
        
        self.space.shapes[0].body.apply_impulse_at_local_point((self.impulse,0))
        
        self.max_height_reached = None
        self.max_potential_energy = None
        
        
        self.draw_options = pymunk.pygame_util.DrawOptions(self.screen)
        self.draw_options.constraint_color = pygame.color.THECOLORS["brown"]
                
        self.reached_apex = False
    

In [10]:
def main():
    pendulumSimulation = simulation()
    pendulumSimulation.start_simulation()
    
    pygame.event.set_allowed([QUIT, KEYDOWN])
    
    
    bullet_mass = 1
    block_mass = 50
    while True:
        for event in pygame.event.get():
            if event.type == QUIT:
                sys.exit(0)
                
            elif event.type == KEYDOWN:
                if event.key == K_ESCAPE:
                    sys.exit(0)
            
                elif event.key == K_e:
                    bullet_mass = input("Set new bullet mass (Default 1): ")
                    block_mass = input("Set new block mass (Default 50): ")
                    pendulumSimulation.restart_simulation(bullet_mass, block_mass)

                        
                elif event.key == K_r:
                    pendulumSimulation.restart_simulation(bullet_mass, block_mass)
                    
                elif event.key == K_f:
                    last_bullet = add_bullet(pendulumSimulation.space)
                    last_bullet.body.apply_impulse_at_local_point((pendulumSimulation.impulse,0))

        pendulumSimulation.run_simulation()
        print(pendulumSimulation.space.shapes, pendulumSimulation.space.bodies)

In [11]:
if __name__ == '__main__':
    sys.exit(main())

([<pymunk.shapes.Circle object at 0x112b6fb10>, <pymunk.shapes.Poly object at 0x112b6f690>], [Body(50.0, 208333.33333333334, Body.DYNAMIC), Body(1.0, 50.0, Body.DYNAMIC)])
([<pymunk.shapes.Circle object at 0x112b6fb10>, <pymunk.shapes.Poly object at 0x112b6f690>], [Body(50.0, 208333.33333333334, Body.DYNAMIC), Body(1.0, 50.0, Body.DYNAMIC)])
([<pymunk.shapes.Circle object at 0x112b6fb10>, <pymunk.shapes.Poly object at 0x112b6f690>], [Body(50.0, 208333.33333333334, Body.DYNAMIC), Body(1.0, 50.0, Body.DYNAMIC)])
([<pymunk.shapes.Circle object at 0x112b6fb10>, <pymunk.shapes.Poly object at 0x112b6f690>], [Body(50.0, 208333.33333333334, Body.DYNAMIC), Body(1.0, 50.0, Body.DYNAMIC)])
([<pymunk.shapes.Circle object at 0x112b6fb10>, <pymunk.shapes.Poly object at 0x112b6f690>], [Body(50.0, 208333.33333333334, Body.DYNAMIC), Body(1.0, 50.0, Body.DYNAMIC)])
([<pymunk.shapes.Circle object at 0x112b6fb10>, <pymunk.shapes.Poly object at 0x112b6f690>], [Body(50.0, 208333.33333333334, Body.DYNAMIC), 

([<pymunk.shapes.Poly object at 0x112b6f690>], [Body(51.0, 50.0, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f690>], [Body(51.0, 50.0, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f690>], [Body(51.0, 50.0, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f690>], [Body(51.0, 50.0, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f690>], [Body(51.0, 50.0, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f690>], [Body(51.0, 50.0, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f690>], [Body(51.0, 50.0, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f690>], [Body(51.0, 50.0, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f690>], [Body(51.0, 50.0, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f690>], [Body(51.0, 50.0, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f690>], [Body(51.0, 50.0, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f690>], [Body(51.0, 50.0, Body.DYNAMIC)])
([<pymunk.shapes.Poly object

([<pymunk.shapes.Poly object at 0x112b6f690>], [Body(51.0, 50.0, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f690>], [Body(51.0, 50.0, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f690>], [Body(51.0, 50.0, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f690>], [Body(51.0, 50.0, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f690>], [Body(51.0, 50.0, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f690>], [Body(51.0, 50.0, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f690>], [Body(51.0, 50.0, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f690>], [Body(51.0, 50.0, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f690>], [Body(51.0, 50.0, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f690>], [Body(51.0, 50.0, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f690>], [Body(51.0, 50.0, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f690>], [Body(51.0, 50.0, Body.DYNAMIC)])
([<pymunk.shapes.Poly object

([<pymunk.shapes.Poly object at 0x112b6f850>], [Body(51.0, 208333.33333333334, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f850>], [Body(51.0, 208333.33333333334, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f850>], [Body(51.0, 208333.33333333334, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f850>], [Body(51.0, 208333.33333333334, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f850>], [Body(51.0, 208333.33333333334, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f850>], [Body(51.0, 208333.33333333334, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f850>], [Body(51.0, 208333.33333333334, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f850>], [Body(51.0, 208333.33333333334, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f850>], [Body(51.0, 208333.33333333334, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f850>], [Body(51.0, 208333.33333333334, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f850>], [Bo

([<pymunk.shapes.Poly object at 0x112b6f850>], [Body(52.0, 208333.33333333334, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f850>], [Body(52.0, 208333.33333333334, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f850>], [Body(52.0, 208333.33333333334, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f850>], [Body(52.0, 208333.33333333334, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f850>], [Body(52.0, 208333.33333333334, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f850>], [Body(52.0, 208333.33333333334, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f850>], [Body(52.0, 208333.33333333334, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f850>], [Body(52.0, 208333.33333333334, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f850>], [Body(52.0, 208333.33333333334, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f850>], [Body(52.0, 208333.33333333334, Body.DYNAMIC)])
([<pymunk.shapes.Poly object at 0x112b6f850>], [Bo

...

([<pymunk.shapes.Circle object at 0x112b6ff50>, <pymunk.shapes.Circle object at 0x112b71210>, <pymunk.shapes.Circle object at 0x112b71350>, <pymunk.shapes.Circle object at 0x112b6fd90>], [Body(1.0, 50.0, Body.DYNAMIC), Body(1.0, 50.0, Body.DYNAMIC), Body(1.0, 50.0, Body.DYNAMIC), Body(0.0, 0.0, Body.DYNAMIC)])
([<pymunk.shapes.Circle object at 0x112b6ff50>, <pymunk.shapes.Circle object at 0x112b71210>, <pymunk.shapes.Circle object at 0x112b71350>, <pymunk.shapes.Circle object at 0x112b6fd90>], [Body(1.0, 50.0, Body.DYNAMIC), Body(1.0, 50.0, Body.DYNAMIC), Body(1.0, 50.0, Body.DYNAMIC), Body(0.0, 0.0, Body.DYNAMIC)])
([<pymunk.shapes.Circle object at 0x112b6ff50>, <pymunk.shapes.Circle object at 0x112b71210>, <pymunk.shapes.Circle object at 0x112b71350>, <pymunk.shapes.Circle object at 0x112b6fd90>], [Body(1.0, 50.0, Body.DYNAMIC), Body(1.0, 50.0, Body.DYNAMIC), Body(1.0, 50.0, Body.DYNAMIC), Body(0.0, 0.0, Body.DYNAMIC)])
([<pymunk.shapes.Circle object at 0x112b6ff50>, <pymunk.shapes.C

From cffi callback <function f5 at 0x112b50f50>:
Traceback (most recent call last):
  File "/Users/alecx/anaconda3/envs/physics/lib/python2.7/site-packages/pymunk/space_debug_draw_options.py", line 93, in f5
    self.draw_dot(size, Vec2d._fromcffi(pos), self._c(color))
  File "/Users/alecx/anaconda3/envs/physics/lib/python2.7/site-packages/pymunk/pygame_util.py", line 166, in draw_dot
    p = to_pygame(pos, self.surface)
  File "/Users/alecx/anaconda3/envs/physics/lib/python2.7/site-packages/pymunk/pygame_util.py", line 183, in to_pygame
    return int(p[0]), surface.get_height()-int(p[1])
ValueError: cannot convert float NaN to integer
From cffi callback <function f2 at 0x112b5fe60>:
Traceback (most recent call last):
  File "/Users/alecx/anaconda3/envs/physics/lib/python2.7/site-packages/pymunk/space_debug_draw_options.py", line 71, in f2
    Vec2d._fromcffi(a), Vec2d._fromcffi(b), self._c(color))
  File "/Users/alecx/anaconda3/envs/physics/lib/python2.7/site-packages/pymunk/pygame_u

([<pymunk.shapes.Circle object at 0x112b6ff50>, <pymunk.shapes.Circle object at 0x112b71210>, <pymunk.shapes.Circle object at 0x112b71350>, <pymunk.shapes.Circle object at 0x112b6fd90>], [Body(1.0, 50.0, Body.DYNAMIC), Body(1.0, 50.0, Body.DYNAMIC), Body(1.0, 50.0, Body.DYNAMIC), Body(0.0, 0.0, Body.DYNAMIC)])
([<pymunk.shapes.Circle object at 0x112b6ff50>, <pymunk.shapes.Circle object at 0x112b71210>, <pymunk.shapes.Circle object at 0x112b71350>, <pymunk.shapes.Circle object at 0x112b6fd90>], [Body(1.0, 50.0, Body.DYNAMIC), Body(1.0, 50.0, Body.DYNAMIC), Body(1.0, 50.0, Body.DYNAMIC), Body(0.0, 0.0, Body.DYNAMIC)])
([<pymunk.shapes.Circle object at 0x112b6ff50>, <pymunk.shapes.Circle object at 0x112b71210>, <pymunk.shapes.Circle object at 0x112b71350>, <pymunk.shapes.Circle object at 0x112b6fd90>], [Body(1.0, 50.0, Body.DYNAMIC), Body(1.0, 50.0, Body.DYNAMIC), Body(1.0, 50.0, Body.DYNAMIC), Body(0.0, 0.0, Body.DYNAMIC)])
([<pymunk.shapes.Circle object at 0x112b6ff50>, <pymunk.shapes.C

From cffi callback <function f5 at 0x112b50f50>:
Traceback (most recent call last):
  File "/Users/alecx/anaconda3/envs/physics/lib/python2.7/site-packages/pymunk/space_debug_draw_options.py", line 93, in f5
    self.draw_dot(size, Vec2d._fromcffi(pos), self._c(color))
  File "/Users/alecx/anaconda3/envs/physics/lib/python2.7/site-packages/pymunk/pygame_util.py", line 166, in draw_dot
    p = to_pygame(pos, self.surface)
  File "/Users/alecx/anaconda3/envs/physics/lib/python2.7/site-packages/pymunk/pygame_util.py", line 183, in to_pygame
    return int(p[0]), surface.get_height()-int(p[1])
ValueError: cannot convert float NaN to integer
From cffi callback <function f2 at 0x112b5fe60>:
Traceback (most recent call last):
  File "/Users/alecx/anaconda3/envs/physics/lib/python2.7/site-packages/pymunk/space_debug_draw_options.py", line 71, in f2
    Vec2d._fromcffi(a), Vec2d._fromcffi(b), self._c(color))
  File "/Users/alecx/anaconda3/envs/physics/lib/python2.7/site-packages/pymunk/pygame_u

KeyboardInterrupt: 